# Data Engineering Individual Assignment

In [ ]:
# This project uses a customer support ticket dataset to simulate real-world enterprise queries.
# Data is ingested from multiple formats (CSV, SQLite, PDF), embedded using OpenAI models, and stored in ChromaDB.
# The goal is to build a RAG pipeline that enables natural language question-answering over this multi-source data.

In [2]:
# ----------------------------------
# Importing the necessary libraries
# ----------------------------------

# Install libraries
!pip install -U langchain langchain-community chromadb pypdf tiktoken sentence-transformers

# Import libraries
import os
import sqlite3
import pandas as pd
from typing import List
from langchain.docstore.document import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 62.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.6/340.6 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.8 MB/s eta 0:00:00


In [3]:
# --------------------------
# Load and Clean the Dataset
# --------------------------

# Load dataset
df = pd.read_csv("customer_support_tickets.csv")
df.head()

,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


In [6]:
# Standardize column names
df.columns = df.columns.str.strip().str.lower()

# Drop duplicates and missing values in important columns
df.drop_duplicates(inplace=True)
df.dropna(subset=["ticket description", "resolution"], inplace=True)

# Lowercase the text columns
df["ticket description"] = df["ticket description"].str.lower()
df["resolution"] = df["resolution"].str.lower()

# Preview cleaned data
df[["ticket description", "resolution"]].head()

,ticket description,resolution
2,i'm facing a problem with my {product_purchase...,case maybe show recently my computer follow.
3,i'm having an issue with the {product_purchase...,try capital clearly never color toward story.
4,i'm having an issue with the {product_purchase...,west decision evidence bit.
10,i'm having an issue with the {product_purchase...,measure tonight surface feel forward.
11,i'm having an issue with the {product_purchase...,measure there house management pick knowledge ...


In [8]:
# ---------------------------
# Export in Multiple Formats
# ---------------------------

import os
os.makedirs("data", exist_ok=True)

# Save as CSV

df.to_csv("data/support_tickets.csv", index=False)

In [9]:
# Save as SQLite

import sqlite3
conn = sqlite3.connect("data/support_tickets.db")
df.to_sql("tickets", conn, if_exists="replace", index=False)
conn.close()

In [11]:
# Convert Sample to PDF

!pip install fpdf
from fpdf import FPDF

pdf = FPDF()
pdf.add_page()
pdf.set_font("Arial", size=12)

# Add 10 tickets
for i in range(10):
    desc = df.iloc[i]["ticket description"]
    res = df.iloc[i]["resolution"]
    pdf.multi_cell(0, 10, f"Ticket {i+1}\nDescription: {desc}\nResolution: {res}\n")

pdf.output("data/support_summary.pdf")

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=5a14b8495d2c7acd0c008eb67a93d3000196f195d7617d84c924a149c23951d0
  Stored in directory: /root/.cache/pip/wheels/65/4f/66/bbda9866da446a72e206d6484cd97381cbc7859a7068541c36
Successfully built fpdf


''

In [12]:
# Verifying all formats

import os
import sqlite3
import pandas as pd

# File paths
csv_path = "data/support_tickets.csv"
db_path = "data/support_tickets.db"
pdf_path = "data/support_summary.pdf"

# 1. Check if files exist
print("File existence check:")
print(f"- CSV: {'Found' if os.path.exists(csv_path) else 'Missing'}")
print(f"- SQLite DB: {'Found' if os.path.exists(db_path) else 'Missing'}")
print(f"- PDF: {'Found' if os.path.exists(pdf_path) else 'Missing'}")

# 2. Preview CSV
print("\n CSV Preview:")
df_csv = pd.read_csv(csv_path)
print(df_csv[["ticket description", "resolution"]].head())

# 3. Preview SQLite contents
print("\n SQLite Preview:")
conn = sqlite3.connect(db_path)
df_sqlite = pd.read_sql("SELECT * FROM tickets LIMIT 5", conn)
conn.close()
print(df_sqlite[["ticket description", "resolution"]])

# 4. Confirm PDF size (optional)
print(f"\n PDF File Size: {os.path.getsize(pdf_path)} bytes")

File existence check:
- CSV: Found
- SQLite DB: Found
- PDF: Found

 CSV Preview:
                                  ticket description  \
0  i'm facing a problem with my {product_purchase...   
1  i'm having an issue with the {product_purchase...   
2  i'm having an issue with the {product_purchase...   
3  i'm having an issue with the {product_purchase...   
4  i'm having an issue with the {product_purchase...   

                                          resolution  
0       case maybe show recently my computer follow.  
1      try capital clearly never color toward story.  
2                        west decision evidence bit.  
3              measure tonight surface feel forward.  
4  measure there house management pick knowledge ...  

 SQLite Preview:
                                  ticket description  \
0  i'm facing a problem with my {product_purchase...   
1  i'm having an issue with the {product_purchase...   
2  i'm having an issue with the {product_purchase...   
3  i'm ha

In [13]:
# --------------------
# Load into LangChain
# --------------------

!pip install langchain langchain-community
!pip install sqlalchemy
!pip install pdfminer.six

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 36.6 MB/s eta 0:00:00


In [14]:
# Import the document loaders

from langchain_community.document_loaders import CSVLoader, SQLDatabaseLoader, PDFMinerLoader
from langchain_community.utilities.sql_database import SQLDatabase

In [15]:
# Load CSV into documents

csv_docs = CSVLoader(file_path="data/support_tickets.csv").load()
print(f"Loaded {len(csv_docs)} documents from CSV.")

Loaded 2769 documents from CSV.


In [16]:
# Load SQLite into documents

db = SQLDatabase.from_uri("sqlite:///data/support_tickets.db")
sqlite_loader = SQLDatabaseLoader(query="SELECT * FROM tickets", db=db)
sqlite_docs = sqlite_loader.load()
print(f"Loaded {len(sqlite_docs)} documents from SQLite.")

Loaded 2769 documents from SQLite.


In [18]:
# Load PDF into documents

!pip install pymupdf

from langchain_community.document_loaders import PyMuPDFLoader

pdf_docs = PyMuPDFLoader("data/support_summary.pdf").load()
print(f"Loaded {len(pdf_docs)} documents from PDF using PyMuPDF.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 55.9 MB/s eta 0:00:00
Loaded 3 documents from PDF using PyMuPDF.


In [19]:
# Combine all into one list

all_docs = csv_docs + sqlite_docs + pdf_docs
print(f"\nTotal documents loaded: {len(all_docs)}")

# Preview of a few documents
for i, doc in enumerate(all_docs[:3]):
    print(f"\n--- Doc {i+1} ---")
    print("Metadata:", doc.metadata)
    print("Content:", doc.page_content[:200])  # first 200 chars


Total documents loaded: 5541

--- Doc 1 ---
Metadata: {'source': 'data/support_tickets.csv', 'row': 0}
Content: ticket id: 3
customer name: Christopher Robbins
customer email: gonzalestracy@example.com
customer age: 48
customer gender: Other
product purchased: Dell XPS
date of purchase: 2020-07-14
ticket type: 

--- Doc 2 ---
Metadata: {'source': 'data/support_tickets.csv', 'row': 1}
Content: ticket id: 4
customer name: Christina Dillon
customer email: bradleyolson@example.org
customer age: 27
customer gender: Female
product purchased: Microsoft Office
date of purchase: 2020-11-13
ticket t

--- Doc 3 ---
Metadata: {'source': 'data/support_tickets.csv', 'row': 2}
Content: ticket id: 5
customer name: Alexander Carroll
customer email: bradleymark@example.com
customer age: 67
customer gender: Female
product purchased: Autodesk AutoCAD
date of purchase: 2020-02-04
ticket t


In [26]:
# --------------------------------
# Embed Documents with HuggingFace
# --------------------------------

!pip install sentence-transformers chromadb

In [27]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# Load the embedding model
embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Store in ChromaDB
vectorstore = Chroma.from_documents(
    documents=all_docs,
    embedding=embedding_function,
    persist_directory="./chroma_db"  # Folder to store vector DB
)

# Save to disk
vectorstore.persist()

print("Documents embedded and stored in ChromaDB using HuggingFace.")

<ipython-input-27-93696a905b72>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  war

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Documents embedded and stored in ChromaDB using HuggingFace.


<ipython-input-27-93696a905b72>:15: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [28]:
# -------------
# Querying RAG
# -------------

# Re-load the vector store from disk

from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings

# Load your stored Chroma DB with same embedding function
embedding_function = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = Chroma(
    persist_directory="./chroma_db",
    embedding_function=embedding_function
)

<ipython-input-28-7751626650a4>:13: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectorstore = Chroma(


In [30]:
# Perform a simple search query

query = "How are billing issues typically resolved?"
docs = vectorstore.similarity_search(query, k=3)

print(f"Top results for: '{query}'\n")
for i, doc in enumerate(docs, 1):
    print(f"Result {i}:\n{doc.page_content[:500]}\n{'-'*60}")

Top results for: 'How are billing issues typically resolved?'

Result 1:
ticket id: 1648
customer name: Brandon Reese
customer email: zmurphy@example.net
customer age: 46
customer gender: Male
product purchased: Canon DSLR Camera
date of purchase: 2021-04-01
ticket type: Billing inquiry
ticket subject: Network problem
ticket description: i'm having an issue with the {product_purchased}. please assist.


my question to you are, are you aware of any other problems in the u.s., or elsewhere, and are there any specific problems that you would the issue i'm facing is inte
------------------------------------------------------------
Result 2:
ticket id: 1648
customer name: Brandon Reese
customer email: zmurphy@example.net
customer age: 46
customer gender: Male
product purchased: Canon DSLR Camera
date of purchase: 2021-04-01
ticket type: Billing inquiry
ticket subject: Network problem
ticket description: i'm having an issue with the {product_purchased}. please assist.


my question to you ar